In [17]:
!python -V

Python 3.9.12


In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import pickle

In [19]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment tracking/mlruns/2', creation_time=1764053820825, experiment_id='2', last_update_time=1764053820825, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [21]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [22]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [23]:

len(df_train), len(df_val)

(73908, 61921)

In [24]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [25]:

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [26]:

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [27]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred,squared=False)

7.479562160810692

In [28]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [29]:
with mlflow.start_run():
    
    mlflow.set_tag("developer","madhu")

    mlflow.log_param("train-data-path","../data/green_tripdata_2021-01.parque")
    mlflow.log_param('valid-data-path',"../data/green_tripdata_2021-02.parquet")
    alpha=0.1
    mlflow.log_param("alpha",alpha)
    lr=Lasso(alpha)
    lr.fit(X_train,y_train)
    y_pred=lr.predict(X_val)

    rmse=mean_squared_error(y_val,y_pred,squared=False)
    mlflow.log_metric("rmse",rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin",artifact_path="model_pickle")


In [30]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [31]:

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [32]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [33]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=Trials()
)

  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:45:45] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.28887                          
[1]	validation-rmse:6.79774                          
[2]	validation-rmse:6.71454                          
[3]	validation-rmse:6.68684                          
[4]	validation-rmse:6.67942                          
[5]	validation-rmse:6.67195                          
[6]	validation-rmse:6.66618                          
[7]	validation-rmse:6.66234                          
[8]	validation-rmse:6.65890                          
[9]	validation-rmse:6.65721                          
[10]	validation-rmse:6.65633                         
[11]	validation-rmse:6.65507                         
[12]	validation-rmse:6.65486                         
[13]	validation-rmse:6.65377                         
[14]	validation-rmse:6.65242                         
[15]	validation-rmse:6.65189                         
[16]	validation-rmse:6.65057                         
[17]	validation-rmse:6.64975                         
[18]	validation-rmse:6.64776

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:46:33] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.60970                                                   
[1]	validation-rmse:8.18943                                                   
[2]	validation-rmse:7.45142                                                   
[3]	validation-rmse:7.07754                                                   
[4]	validation-rmse:6.88876                                                   
[5]	validation-rmse:6.78606                                                   
[6]	validation-rmse:6.72898                                                   
[7]	validation-rmse:6.69454                                                   
[8]	validation-rmse:6.67384                                                   
[9]	validation-rmse:6.65840                                                   
[10]	validation-rmse:6.64898                                                  
[11]	validation-rmse:6.64226                                                  
[12]	validation-rmse:6.63794                        

In [35]:
import mlflow

run_id = "cd11f75b75e4478ca4a970b6b52ed4ec"
model_uri = f"runs:/{run_id}/model_pickle"

mlflow.register_model(
    model_uri=model_uri,
    name="NYC-Taxi-Model"
)


Successfully registered model 'NYC-Taxi-Model'.


MlflowException: Unable to find a logged_model with artifact_path model_pickle under run cd11f75b75e4478ca4a970b6b52ed4ec